## 필요사항 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.0 MB/s 
     |████████████████████████████████| 163 kB 81.7 MB/s 
     |████████████████████████████████| 7.6 MB 70.3 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.7 MB/s 
     |████████████████████████████████| 115 kB 75.8 MB/s 
     |████████████████████████████████| 212 kB 79.3 MB/s 
     |████████████████████████████████| 127 kB 82.3 MB/s 
     |████████████████████████████████| 115 kB 71.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 69.6 MB/s 
     |████████████████████████████████| 166 kB 78.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 82.4 MB/s 
     |████████████████████████████████| 162 kB 73.8 MB/s 
     |████████████████████████████████| 162 kB 73.5 MB/s 
     |████████████████████████████████| 158 kB 79.9 MB/s 
     |████████████████████████████████| 157 kB 75.3 MB/s 
     |████████████████████████████████| 157 kB 68.1 MB/s 
     |████████████████████████████████| 157 kB 82.3 MB/s 
     |████████████████████████████████| 157 kB 70.0 MB/s 
     |████████████████████████████████| 157 kB 80.7 MB/s 
     |████████████████████████████████| 157 kB 64.9 MB/s 
     |████████████████████████████████| 157 kB 66.3 MB/s 
     |████████████████████████████████| 156 kB 83.9 MB/s 


In [4]:
!pip install bertviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 157 kB 4.9 MB/s 
     |████████████████████████████████| 1.3 MB 69.9 MB/s 
     |████████████████████████████████| 132 kB 69.1 MB/s 
     |████████████████████████████████| 9.3 MB 72.2 MB/s 
     |████████████████████████████████| 79 kB 9.1 MB/s 


In [5]:
!kaggle datasets download -d minjaechoi99/aihub-groom

 87% 33.0M/38.1M [00:00<00:00, 132MB/s] 
100% 38.1M/38.1M [00:00<00:00, 135MB/s]


In [6]:
!unzip /content/aihub-groom.zip

Archive:  /content/aihub-groom.zip
  inflating: data/Training/talksets-train-1.json  
  inflating: data/Training/talksets-train-2.json  
  inflating: data/Training/talksets-train-3.json  
  inflating: data/Training/talksets-train-4.json  
  inflating: data/Training/talksets-train-5.json  
  inflating: data/Validation/talksets-train-6.json  


## 데이터 전처리

In [7]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
import torch
import torch.optim as optim

import numpy as np
import tqdm as tqdm

from transformers import(
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule,
    DefaultDataCollator,
    AdamW
)

from bertviz import model_view

## readjson

In [9]:
from pathlib import Path
import json, re
def read_aihub(path):
    path = Path(path)
    with open(path, 'rb') as f:
        print(f)
        squad_dict = json.load(f)
    texts = []
    intensities = []
    for group in squad_dict:
        for passage in group['sentences']:
            text = passage['origin_text']
            text = re.sub('#.*?#', '[UNK]', text)
            intensity = passage['intensity']
            if intensity == 1:
                continue
            texts.append(text)
            if intensity < 1:
                intensity = 0
            else:
                intensity = 1
            intensities.append(intensity)

    return texts ,intensities

In [10]:
import pandas as pd

file_path = '/content/data/Validation/talksets-train-6.json'

texts , intensities = read_aihub(file_path)
test_df = pd.DataFrame(list(zip(texts , intensities)),
               columns =['texts','intensities'])

<_io.BufferedReader name='/content/data/Validation/talksets-train-6.json'>


In [11]:
from datasets import Dataset

test_data = Dataset.from_pandas(test_df)

In [12]:
test_data

Dataset({
    features: ['texts', 'intensities'],
    num_rows: 37319
})

## tokenize

In [13]:
model_name = 'beomi/KcELECTRA-base-v2022'
max_length = 128

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
def tokenizeWithoutLabel(data):
    tokenized_datas = tokenizer(
        data['texts'],
        max_length=max_length,
        padding="max_length",
        truncation="only_second"
    )
    return tokenized_datas

In [16]:
test_tokenized_datasets = test_data.map(tokenizeWithoutLabel, batched=True, remove_columns=test_data.column_names)

  0%|          | 0/38 [00:00<?, ?ba/s]

In [17]:
test_tokenized_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 37319
})

## test

In [18]:
import wandb
run = wandb.init()

wandb: Currently logged in as: mjchoi. Use `wandb login --relogin` to force relogin


In [19]:
from tqdm.auto import tqdm
def get_prob(model, test_loader):
    model = model.to(device)
    pred = []
    error = 0
    special_tokens = set(tokenizer.all_special_ids)
    softmax = torch.nn.Softmax()
    for i, id_inputs in enumerate(tqdm(test_loader)):
        torch.cuda.empty_cache()
        inputs = {}
        inputs['input_ids'] = id_inputs['input_ids'].to(device)
        inputs['token_type_ids'] = id_inputs['token_type_ids'].to(device)
        inputs['attention_mask'] = id_inputs['attention_mask'].to(device)
        outputs = model(**inputs)
        pred += [0 if logit[0] > logit[1] else 1 for logit in outputs.logits]
    return pred

In [20]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

artifact1 = run.use_artifact('groom2team/pj3_finetune_classifier/pytorch_finetuned:v0', type='model')
# test 할 모델 위치를 use_artifact에 넣는다.
artifact_dir1 = artifact1.download()

torch.cuda.empty_cache()

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.load_state_dict(torch.load(artifact_dir1+'/best_model_at_end/pytorch_model.bin'))

data_collator = DefaultDataCollator(return_tensors="pt")
test_loader = torch.utils.data.DataLoader(test_tokenized_datasets, batch_size=64,
                                        shuffle=False, collate_fn=data_collator,
                                        num_workers=0)

device = torch.device('cuda')

pred = get_prob(model, test_loader)
predictions = test_df.drop(['texts'], axis = 1)
predictions['pred'] = pred
predictions.rename(columns = {'intensities':'answer'},inplace=True)

data=predictions

ans=data['answer']
pre=data['pred']
print('score acc, recall, precision, f1')
print(accuracy_score(ans, pre))	
print(recall_score(ans, pre))
print(precision_score(ans, pre))
print(f1_score(ans, pre))
print()
predictions.to_csv(f'predictions.csv', index = False)

wandb: Downloading large artifact pytorch_finetuned:v0, 489.19MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:5.2


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.out_proj.weight

  0%|          | 0/584 [00:00<?, ?it/s]

nth score acc, recall, precision, f1
0.8196897022964174
0.9707216494845361
0.7255834434169969
0.8304397127378104

